In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Transformer

/content/drive/My Drive/Colab Notebooks/Transformer


# ライブラリ読み込み

In [3]:
%%bash 
pip install SudachiPy
pip install SudachiDict-full
pip install sortedcontainers==2.1.0

  Created wheel for SudachiPy: filename=SudachiPy-0.5.1-cp36-cp36m-linux_x86_64.whl size=871015 sha256=eace931f9e35536f97af6bbdb3384801866ba4af015f3ce8e2f8be4345b2cd8f
  Stored in directory: /root/.cache/pip/wheels/a5/fb/54/82b45dd3d48476dfe6d055da5dee9d5fa1c1941368f9568e90
Successfully built SudachiPy
  Found existing installation: sortedcontainers 2.3.0
    Uninstalling sortedcontainers-2.3.0:
      Successfully uninstalled sortedcontainers-2.3.0
  Created wheel for SudachiDict-full: filename=SudachiDict_full-20201223.post1-cp36-none-any.whl size=126956073 sha256=2364dd4fdd7a142eace25dbc1cab156a6585c9b8e0e8c82295fce6b72889c642
  Stored in directory: /root/.cache/pip/wheels/e4/b4/b1/8e5c8f1bb9052bb69259c9674b5cfb09a3bdceb01cb009e91a
Successfully built SudachiDict-full


In [4]:
!sudachipy link -t full

Set the default dictionary to `sudachidict_full`.


In [5]:
!pwd

/content/drive/My Drive/Colab Notebooks/Transformer


ランタイムを再駆動する

In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Transformer

In [1]:
from sudachipy import tokenizer 
from sudachipy import dictionary
tokenizer_obj = dictionary.Dictionary().create()

In [3]:
import pandas as pd
import re
import codecs
import copy

# データ読み込み

In [10]:

corpus_path = './DATA/Kesennuma.csv'
df = pd.read_csv(corpus_path)
mode = tokenizer.Tokenizer.SplitMode.C
df['A'] = ""
print('**** Amount of data ****')
print(df)
print('\n')
print('**** Amount of data ****')
#for index, row in df.iterrows():
    #print(row['項目名'])


**** Amount of data ****
    市区町村             項目名        回答語形 A 
0    蟹田町    鏡（かがみ）：ガ行鼻濁音    ［ka?ami］   
1    蟹田町  袴（はかま）：カ行子音有声化    ［hakama］   
2    蟹田町      肌（はだ）：ダ行子音      ［hada］   
3    蟹田町   旗（はた）：タ行子音有声化      ［hata］   
4    蟹田町       座布団：ザ・ダ交替  ［dzab?to?］   
..   ...             ...         ... ..
261  蟹田町     〈もしもし〉○○でした       ○○デス‥   
262  蟹田町      ありがとう：買い物時   マン＾ダクルハネノ   
263  蟹田町      ありがとう：買い物時        サイナラ   
264  蟹田町       ありがとう：家庭内          ＮＲ   
265  蟹田町            ではまた         ドーモ   

[266 rows x 4 columns]


**** Amount of data ****


# 前処理

In [14]:
c_path = './DATA/Kessen.txt'

with open(c_path, 'a') as f:
    for index, row in df.iterrows():
        s = re.sub('[〈]+.*[〉]+', '', row['項目名'], 2)
        s = re.sub('[（]+.{1,7}[）]+', '', s, 2)
        s = re.sub('[（]+.{1,8}[・]+.{1,8}[）]+', '', s, 2)
        s = re.sub('[：].*[音意語].*', '', s, 2)
        s = re.sub('[\u3041-\u309F]??[：]', '', s, 2)
        #print(s)
        tokens =  tokenizer_obj.tokenize( s, mode)
        parsed = ""
        for t in tokens:
            parsed +=  str(t.reading_form())
        if parsed == "" :
            parsed = row['項目名']
        print(parsed,file=f)

# モデル定義

In [15]:
c_path = './DATA/Kessen.txt'

c_df = copy.deepcopy(df)

with open(c_path, 'a') as f:
    for index, row in c_df.iterrows():
        s = re.sub('[〈]+.*[〉]+', '', row['項目名'], 2)
        s = re.sub('[（]+.{1,7}[）]+', '', s, 2)
        s = re.sub('[（]+.{1,8}[・]+.{1,8}[）]+', '', s, 2)
        s = re.sub('[：].*[音意語].*', '', s, 2)
        s = re.sub('[\u3041-\u309F]??[：]', '', s, 2)
        #print(s)
        tokens =  tokenizer_obj.tokenize( s, mode)
        parsed = ""
        for t in tokens:
            parsed +=  str(t.reading_form())
        if parsed == "" :
            parsed = row['項目名']
        row['A'] = parsed
        
        #print(parsed,file=f)
print(c_df.head())

c_df.to_csv('./DATA/Tugaru3.csv', index=False)

  市区町村             項目名        回答語形            A
0  蟹田町    鏡（かがみ）：ガ行鼻濁音    ［ka?ami］          カガミ
1  蟹田町  袴（はかま）：カ行子音有声化    ［hakama］          ハカマ
2  蟹田町      肌（はだ）：ダ行子音      ［hada］           ハダ
3  蟹田町   旗（はた）：タ行子音有声化      ［hata］           ハタ
4  蟹田町       座布団：ザ・ダ交替  ［dzab?to?］  ザブトンザ・ダコウタイ


In [16]:
c_path = './DATA/Kessen.txt'

c_df2 = copy.deepcopy(df)

with open(c_path, 'a') as f:
    for index, row in c_df2.iterrows():
        s = re.sub('[〈]+.*[〉]+', '', row['項目名'], 2)
        s = re.sub('[（]+.{1,7}[）]+', '', s, 2)
        s = re.sub('[（]+.{1,8}[・]+.{1,8}[）]+', '', s, 2)
        s = re.sub('[：].*[音意語].*', '', s, 2)
        s = re.sub('[\u3041-\u309F]??[：]', '', s, 2)
        #print(s)
        tokens =  tokenizer_obj.tokenize( s, mode)
        parsed = ""
        for t in tokens:
            parsed +=  str(t.reading_form())
        if parsed == "" :
            parsed = row['項目名']
        row['A'] = parsed
        
        #print(parsed,file=f)
c_df2 = c_df2.reindex(columns=['A', '回答語形', '市区町村', '項目名'])
print(c_df2.head())

c_df2.to_csv('./DATA/Tugaru3.tsv', sep='\t', index=False)

             A        回答語形 市区町村             項目名
0          カガミ    ［ka?ami］  蟹田町    鏡（かがみ）：ガ行鼻濁音
1          ハカマ    ［hakama］  蟹田町  袴（はかま）：カ行子音有声化
2           ハダ      ［hada］  蟹田町      肌（はだ）：ダ行子音
3           ハタ      ［hata］  蟹田町   旗（はた）：タ行子音有声化
4  ザブトンザ・ダコウタイ  ［dzab?to?］  蟹田町       座布団：ザ・ダ交替


# 学習実行